In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**MAKING SENSE OF THE DATA:**
* WE HAVE ACQUIRED DATA FROM TWO DIFFERENT DATASETS INCLUDING:
* RESUMES DATASET(KAGGLE)
* RESUMES ENTITIES FOR NER (KAGGLE)


PROPERTIES OF THE DATA:

    THE DATASET THAT WE MANAGED TO  ACQUIRE HAD THE WHOLE RESUME OF THE CANDIDATES AS WELL AS BROAD CATEGORY THAT MAY FALL IN

In [5]:
import pandas as pd
df1 = pd.read_csv('./resume_dataset2.csv', usecols = ['Category','Resume'])
df = pd.DataFrame(df1)
df


,Category,Resume
0,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...
1214,Aviation,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1215,Aviation,b'Corporate Flight Attendant Resume\nCAITLIN F...
1216,Aviation,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1217,Aviation,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


In [6]:
df2 = pd.read_csv('./resume_dataset.csv')
dff = pd.DataFrame(df2)
df = df.append(dff)
df

,Category,Resume
0,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...
164,Testing,Computer Skills: â¢ Proficient in MS office (...
165,Testing,â Willingness to accept the challenges. â ...
166,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
167,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [7]:
allCategories = df["Category"].unique()
allresumes = df["Resume"].unique()

print("unqiue categories")
print(len(allCategories))
print("unique resumes")
print(len(allresumes))


unqiue categories
46
unique resumes
1257


OUR DATASET HAS 1257 COLUMNS WITH 46 UNIQUE CATEGORIES WHICH INCLUDE ADVOCATE, CIVIL, MECHANICAL ETC


**PROCESSING THE DATA**

COMBINING ALL RESUMES OF SAME CATEGORIES, WHICH WOULD LATER HELP IN RECOMMENDING SKILLS

In [8]:
dict = {}

values = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 ")
def remover(my_string = ""):
  for item in my_string:
    if item not in values:
      my_string = my_string.replace(item, "")
  return my_string


for i,j in df.iterrows():
    if j[0] in dict.keys():
        dict[j[0]] = dict[j[0]] + ' ' + j[1];
    else:
        dict[j[0]] = j[1]


for key,value in dict.items():
    dict[key]=remover(value)

**TOKENIZING, STEMMING AND REMOVING STOPWORDS**

In [9]:
import nltk

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

stop_words = set(stopwords.words('english')) 

for key,value in dict.items():
    temp = word_tokenize(value) 
    token = [w for w in temp if not w in stop_words]
    dict[key] = token
    
print(dict["HR"])

['bJohn', 'H', 'Smith', 'PHRn8009915187', 'PO', 'Box', '1673', 'Callahan', 'FL', '32011', 'infogreatresumesfastcomnnApproachable', 'innovator', 'passion', 'Human', 'ResourcesnnSENIOR', 'HUMAN', 'RESOURCES', 'PROFESSIONALnPersonable', 'analytical', 'flexible', 'Senior', 'HR', 'Professional', 'multifaceted', 'expertise', 'Seasoned', 'Benefits', 'Administrator', 'withnextensive', 'experience', 'working', 'highly', 'paid', 'professionals', 'clientrelationshipbased', 'settings', 'Dynamic', 'team', 'leaderncapable', 'analyzing', 'alternatives', 'identifying', 'tough', 'choices', 'communicating', 'total', 'value', 'benefit', 'andncompensation', 'packages', 'senior', 'level', 'executives', 'employeesnnCORE', 'COMPETENCIESnBenefits', 'Administration', 'xe2x80x93', 'Customer', 'Service', 'xe2x80x93', 'Cost', 'Control', 'xe2x80x93', 'Recruiting', 'xe2x80x93', 'Acquisition', 'Management', 'xe2x80x93', 'Compliance', 'ReportingnRetention', 'xe2x80x93', 'Professional', 'Services', 'xe2x80x93', 'Domes

**INITIATE THE LDA**

The latent Dirichlet allocation (LDA) is a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar. For example, if observations are words collected into documents, it posits that each document is a mixture of a small number of topics and that each word's presence is attributable to one of the document's topics. LDA is an example of a topic model and belongs to the machine learning toolbox and in wider sense to the artificial intelligence toolbox.

In [10]:
import gensim
from gensim import corpora, models

processed = [ v for v in dict.values() ]
dictionary_lda = corpora.Dictionary(processed)
dictionary_lda.filter_extremes(no_below=3)
corpus = [dictionary_lda.doc2bow(listoftokens) for listoftokens in processed]

num_topics = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_lda, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_lda.keys()))

CPU times: user 17.5 s, sys: 11.1 s, total: 28.5 s
Wall time: 9.73 s


WITH ALL WORDS COMBINED IN ALL THE RESUMES WE CREATED 20 BUNCH OF WORDS WHERE EACH DENOTE A UNIQUE TOPIC 

In [29]:

t1 = [] 
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(i)
    t1.append(topic)
    print(topic)
    print()

0
0.007*"banking" + 0.006*"bank" + 0.006*"accounting" + 0.006*"Finance" + 0.005*"tax" + 0.004*"loan" + 0.004*"million" + 0.003*"credit" + 0.003*"finance" + 0.003*"transactions"

1
0.004*"call" + 0.003*"xc2xa0tr" + 0.003*"Call" + 0.002*"J" + 0.002*"Association" + 0.002*"center" + 0.002*"ABC" + 0.002*"medical" + 0.001*"S" + 0.001*"xe2x97x8f"

2
0.007*"Less" + 0.005*"NET" + 0.005*"MVC" + 0.005*"Visual" + 0.005*"JavaScript" + 0.005*"Layer" + 0.004*"Framework" + 0.004*"Studio" + 0.004*"Designing" + 0.004*"Net"

3
0.015*"HR" + 0.008*"Human" + 0.006*"Resources" + 0.005*"recruitment" + 0.003*"candidates" + 0.003*"Resource" + 0.003*"managers" + 0.003*"Employee" + 0.003*"benefits" + 0.003*"Recruitment"

4
0.008*"ETL" + 0.006*"Informatica" + 0.003*"analyzing" + 0.003*"mappings" + 0.002*"source" + 0.002*"skillsnxefx82xb7" + 0.002*"xc2xa0tr" + 0.002*"xe2x80xa2" + 0.002*"target" + 0.002*"tables"

5
0.008*"automotive" + 0.006*"Automotive" + 0.004*"repair" + 0.004*"paint" + 0.004*"Technician" + 0.003*

**ONCE WE HAVE THE TOPICS READY WE CAN ALSO TELL WHICH CATEGORY HAS HIGHEST AFFILIATION TO WHICH TOPIC WHICH WILL HELP IN RECOMMENDING THE KEYWOARDS**

In [12]:
i=0
dict4db = {}
for k,v in dict.items():
    #rint(k)
    jt = (lda_model[corpus[i]])
    dict4x = {};
    for j in jt:
        dict4x[j[0]] = j[1];
    for ni in range(10):
        if ni not in dict4x.keys():
            dict4x[ni] = 0;
    dict4db[k] = dict4x;
    i+=1
    
dict4db

{'HR': {3: 0.9999836, 0: 0, 1: 0, 2: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0},
 'Designing': {19: 0.99997914,
  0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0},
 'Managment': {11: 0.97078973,
  18: 0.024042724,
  0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0},
 'Information Technology': {7: 0.025384111,
  16: 0.032764625,
  17: 0.8648853,
  18: 0.076468974,
  0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  8: 0,
  9: 0},
 'Education': {7: 0.7574906,
  14: 0.24250458,
  0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  8: 0,
  9: 0},
 'Advocate': {13: 0.9999912,
  0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0},
 'Business Development': {0: 0.046360295,
  11: 0.06276443,
  18: 0.8193202,
  19: 0.053124007,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0},
 'Health & Fitness': {10: 0.9838831,
  18: 0.015243194,
  0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 

* KEY WOARDS FOR
* HR : accounting + Finance + food + million + annual + Accounting + tax + revenue + managed
* DESIGNING : Art + Designer + Gallery + Fashion + Museum + Adobe + content + brand
* AND SO ON

**GETTING READY TO CLASSIFY ANY GIVEN RESUME INTO GROUPS**

IN THE FOLLOWING CODE WE ARE ASSIGNIG POINTS TO EACH AND EVERY RESUME ACCORDING TO THEIR CLOSENESS TO THE 20 GROUPS WE HAVE MADE

In [37]:
import re


def keyWord (category =""):
    temp = dict4db[category]
    topic = 0
    high =0
    for k,v in temp.items():
        if v > high:
            high = v
            topic = k
    print("skills required for " + category + " are " )
    ans = (t1[topic])
    print(re.sub('[^a-zA-Z ]+', '', ans))
    
keyWord("Health & Fitness")

skills required for Health & Fitness are 
J  LM  Medical  Association  Dawson  Care  Burke  Nutrition  Sports  Hospital


In [13]:
x = []
y = []
for i,j in df.iterrows():
    y.append(j[0])
    x.append(j[1])
print(len(x))
for i in range(len(x)):
    x[i] = remover(x[i])
    x[i] =( word_tokenize(x[i]))
    x[i] = [w for w in x[i] if not w in stop_words]
    x[i] = lda_model[dictionary_lda.doc2bow(x[i])]
    
for i in range(len(x)):
    dumx = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for j in range(len(x[i])):
        dumx[x[i][j][0]] = x[i][j][1]
    x[i] = dumx
    
i = 100
for v in y:
    print(v)
    print(x[i])
    i+=1
    if i>110:
        break

1388
HR
[0, 0, 0, 0, 0, 0, 0, 0.093590915, 0, 0, 0, 0.3420415, 0, 0.08908134, 0, 0, 0.0237348, 0.2785852, 0.17172599, 0]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.99906445, 0, 0, 0, 0, 0, 0, 0, 0]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9976848, 0, 0, 0, 0, 0, 0, 0, 0]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.98441464, 0, 0, 0, 0, 0, 0, 0, 0.013537666]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9579324, 0, 0, 0, 0, 0, 0, 0.040513612, 0]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.95098144, 0, 0, 0, 0, 0.048635032, 0, 0, 0]
HR
[0, 0, 0, 0, 0, 0, 0.10111831, 0, 0, 0, 0, 0.77156425, 0, 0, 0, 0, 0.10914457, 0, 0, 0.016450733]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.99834776, 0, 0, 0, 0, 0, 0, 0, 0]
HR
[0, 0, 0, 0, 0, 0, 0, 0.019341372, 0, 0, 0, 0.9503677, 0, 0, 0, 0, 0, 0, 0.029684262, 0]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.93218565, 0, 0, 0, 0, 0, 0, 0.067016676, 0]
HR
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9991628, 0, 0, 0, 0, 0, 0, 0, 0]


**WE ARE READY NOW TO TRAIN OUR MODEL**
* THE X[ ] VALUES OF EACH RESUMES ARE THEIR POINT OF AFFINITY TOWARDS THE GROUP
* THE Y VALUE IS THEIR ACTUAL GROUP
* HENCE WE ARE USING SUPERVISED LEARNING

WE RAN NUMBER OF MODELS WITH FOLLOWING ACCURACIES:

* WE CHOOSE _____ FOR THIER BEST PERFORMANCE


from sklearn.datasets import load_iris
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,
         random_state=0).fit(x, y)
gpc.score(x, y)

In [15]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)

from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[12  1  0 ...  0  0  0]
 [ 0 16  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  3  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
                         precision    recall  f1-score   support

             Accountant       0.63      0.75      0.69        16
               Advocate       0.94      1.00      0.97        16
           Agricultural       0.00      0.00      0.00         4
                Apparel       0.00      0.00      0.00         1
             Architects       0.80      1.00      0.89         4
                   Arts       0.79      0.85      0.81        13
     Automation Testing       0.00      0.00      0.00         0
             Automobile       0.67      1.00      0.80         4
               Aviation       1.00      1.00      1.00         2
                    BPO       1.00      0.43      0.60         7
                Banking       0.50      1.00      0.67         6
             Blockchain       0.00      0.00      0.00         

/home/ashutosh/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ashutosh/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**NOW WE CAN SEGREGATE ANY RESUME INTO THE BROAD CATEGORIES WITH MUCH SUCCESS**

* NEXT STEP IS UPLOADING ANY GIVEN RESUME
* EXTRACT TEXT FROM IT
* PUT IT INTO ONE OF THE BROAD CATEGORIES

In [16]:
#!pip install PyPDF2

import PyPDF2

OPEN RESUME FROM PDF FILE OBJECT
EXTRACT THE TEXT INTO THE FORM OF STRING USING THE PDF READER GET PAGE FUNCTION

In [38]:
pdfFileObj = open('./demo.pdf', 'rb') 
  
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

num = (pdfReader.numPages) 
  
string = " ";

for i in range(num):
    pageObj = pdfReader.getPage(i)
    print(i)
    string  += (pageObj.extractText()) 

pdfFileObj.close() 

print(string)

0
1
  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
CRAIG MATHEWS 
 
 
CIVI
L ENGINEER
 
Dayjob Ltd, 120 Vyse Street
, 
Birmingham B18 6NF
 
T:  00 44 121 638 0026
 
-
 
E
: 
info@dayjob.com
 
 
W: www.dayjob.com
 
Summary
 
A high performing civil engineer who ha
s considerable experience of working on infrastructure, 
utilities, geotechnical, structural, highways, drainage and flood prevention projects. Craig possesses 

s 
lives. He is committed to providing a high quality service delivery to every client and project he 
works on, and is 
currently 
looking for employment in an open, friendly and professional 
environment where ideas are shared and opportunities seized.
 
Skills
 
Career
 
CIVIL ENGINEER        
                                                                   
               
    
             
May 2007 
-
 
Present
      
Building Surveyors
 
 
R
esponsible for coming up with creative solutions to technical problems to maintain the 
Company's competitive

**PREPROCESS THE STRING AS WE HAD DONE BEFORE**
* **AND PREDICT THE CATEGORY IN WHICH THE RESUME WOULD FALL IN**

In [41]:
string = remover (string)

temp2 = ( word_tokenize(string))
token2 = [w for w in temp2 if not w in stop_words]
toPredict = lda_model[dictionary_lda.doc2bow(token2)]
print(toPredict)
xp = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for value in toPredict:
    xp[value[0]] = value[1]
print(xp)
print(svclassifier.predict([xp]))

[(5, 0.013055094), (11, 0.38432035), (16, 0.14996947), (18, 0.45161542)]
[0, 0, 0, 0, 0, 0.013055094, 0, 0, 0, 0, 0, 0.38432035, 0, 0, 0, 0, 0.14996947, 0, 0.45161542, 0]
['Engineering']


* **FINALLY THE LAST TASK!!!**
* **RECOMMEND SOME GOOD PROFILES**

WE HAVE CREATED A FUNCTION IN WHICH IF YOU GIVE THE NAME OF THE CATEGORY WE FILTTER OUT THE BEST PROFILES FOR YOU:

In [19]:
category = 'Java Developer'

In [24]:
def recommender (category = ""):
    topic=0
    high=0
    ans=0
    top=0
    for k,value in dict4db[category].items():
        if value > high:
            topic = k
            high = value
    print("highest value for " + category)
    print(topic)
    for ii in range(0,len(y)):
            if y[ii] == category:
                if x[ii][topic]>top:
                    top= x[ii][topic]
                    ans = ii
    if ans>1218:
        ans-= 1218
    
    print("best resume of this profile is:")
    #print(ans)
    for k,v in df['Category'][ans].items():
        if v== category:
            for k1,v1 in df['Resume'][ans].items():
                print(remover(v1))

In [42]:
recommender(category)

highest value for DotNet Developer
2
best resume of this profile is:
bSarah CollinsnnProject Manager ResumenDayjob Ltd The Big Peg 120 Vyse Street Birmingham B18 6NFnTel 0870 061 0121nMobile 0777 777 7777nEmail infodayjobcomnPersonal profilenA multiskilled professional with a superb tracknrecord of managing complex functional projects innvarious environments Able to manage stakeholdernexpectations and willing to take full responsibility fornthe delivering of project objectives Sarah is an easyngoing individual who enjoys challenging and diversenroles and is confident working with technical expertsnfrom any industrynnEXPERIENCE  KEY COMPETENCIESnProject Management skillsnxefx82xb7nxefx82xb7nxefx82xb7nxefx82xb7nnPresently looking to join a company that rewardsneffort and initiative whilst at the same timenproviding plenty of progression and developmentnopportunities to its employeesnAcademic qualificationsnBA Hons Project ManagementncnPRNCEn2nAxe2x80x99 Levels Maths C English A Physics A